In [22]:
import pyomo.environ as pe
import matplotlib.pyplot as plt
import numpy as np

In [23]:
#Q3 data
unit_cost = {1: 6, 2: 16, 3: 10}
unit_price = {1: 9, 2: 15}
raw_quality = {1: 0.030, 2: 0.010, 3: 0.020}
demand = {1: 100, 2: 200}
max_quality = {1: 0.025, 2: 0.015}

In [24]:
solver = pe.SolverFactory('baron')

In [25]:
#Q3 P formulation
model3p = pe.ConcreteModel()
model3p.IDXfeeds = pe.RangeSet(3)
model3p.IDXpoolsources = pe.RangeSet(2)
model3p.IDXdirectsupply = {3}
model3p.IDXproducts = pe.RangeSet(2)
# idx of pools and idx of qualities are just 1
model3p.c = pe.Param(model3p.IDXfeeds, initialize=unit_cost)
model3p.d = pe.Param(model3p.IDXproducts, initialize=unit_price)
model3p.C = pe.Param(model3p.IDXfeeds, initialize=raw_quality)
model3p.D = pe.Param(model3p.IDXproducts, initialize=demand)
model3p.PU = pe.Param(model3p.IDXproducts, initialize=max_quality)

model3p.p = pe.Var(within=pe.NonNegativeReals)
# remember no raw mateiral 3 goes to pool
model3p.x = pe.Var(model3p.IDXpoolsources, within=pe.NonNegativeReals)
model3p.y = pe.Var(model3p.IDXproducts, within=pe.NonNegativeReals)
# remember no raw mateiral 1 and 2 go to direct supply
model3p.z = pe.Var(model3p.IDXdirectsupply, model3p.IDXproducts, within=pe.NonNegativeReals)

def obj_rule(m):
    cost =  sum(m.c[i] * m.x[i] for i in m.IDXpoolsources)
    revenue = sum(m.d[j] * m.y[j] for j in m.IDXproducts) + \
                sum((m.d[j] - m.c[i]) * m.z[i,j] for i in m.IDXdirectsupply for j in m.IDXproducts)
    return cost - revenue
model3p.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

# Conservation of flow into and out of the pools
def balance_pool_rule(m):
    return sum(m.x[i] for i in m.IDXpoolsources) == sum(m.y[j] for j in m.IDXproducts)
model3p.balance_pool = pe.Constraint(rule=balance_pool_rule)

# Sales of each product cannot exceed their demands
def demand_rule(m, j):
    return m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply) <= m.D[j]
model3p.demand = pe.Constraint(model3p.IDXproducts, rule=demand_rule)

# Maintains the quality of the product within upper bounds
def quality_rule(m, j):
    return m.p * m.y[j] + sum(m.C[i] * m.z[i, j] for i in m.IDXdirectsupply) <= m.PU[j] * (m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply))
model3p.quality = pe.Constraint(model3p.IDXproducts, rule=quality_rule)

# Balances the quality
def balance_quality_rule(m):
    return sum(m.C[i] * m.x[i] for i in m.IDXpoolsources) == m.p * sum(m.y[j] for j in m.IDXproducts)
model3p.balance_quality = pe.Constraint(rule=balance_quality_rule)

result_obj = solver.solve(model3p, tee=True)
model3p.pprint()
print(result_obj)

 BARON version 23.6.15. Built: OSX-64 Thu Jun 15 21:44:35 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Can Li at Purdue University, canli@purdue.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSQP
 Starting solution is feasible with a value of   0.00000          
 Doing local search
 Preprocessing found feasible solution with value -100.000
 Solving bounding LP
 Starting multi-start local search
 Preprocessing found feasible solution with value -400.000
 Preprocessing found feasible solution with value -400.000
 Preprocessing found feasible solu

In [26]:
#Q3 Q formulation
model3q = pe.ConcreteModel()
model3q.IDXfeeds = pe.RangeSet(3)
model3q.IDXpoolsources = pe.RangeSet(2)
model3q.IDXdirectsupply = {3}
model3q.IDXproducts = pe.RangeSet(2)
# idx of pools and idx of qualities are just 1
model3q.c = pe.Param(model3q.IDXfeeds, initialize=unit_cost)
model3q.d = pe.Param(model3q.IDXproducts, initialize=unit_price)
model3q.C = pe.Param(model3q.IDXfeeds, initialize=raw_quality)
model3q.D = pe.Param(model3q.IDXproducts, initialize=demand)
model3q.PU = pe.Param(model3q.IDXproducts, initialize=max_quality)

model3q.p = pe.Var(within=pe.NonNegativeReals)
# remember no raw mateiral 3 goes to pool
model3q.q = pe.Var(model3q.IDXpoolsources, within=pe.NonNegativeReals)
model3q.y = pe.Var(model3q.IDXproducts, within=pe.NonNegativeReals)
# remember no raw mateiral 1 and 2 go to direct supply
model3q.z = pe.Var(model3q.IDXdirectsupply, model3q.IDXproducts, within=pe.NonNegativeReals)

def obj_rule(m):
    # term1 = sum((sum(m.q[i] * m.y[j] * m.C[i, j] for i in m.IDXpoolsources)) * y[j] for j in m.IDXproducts)
    # term2 = sum(m.c[i] * m.x[i] for i in m.IDXpoolsources)
    cost =  sum(m.c[i] * (m.q[i] * sum(m.y[j] for j in m.IDXproducts)) for i in m.IDXpoolsources)
    revenue = sum(m.d[j] * m.y[j] for j in m.IDXproducts) + \
                sum((m.d[j] - m.c[i]) * m.z[i,j] for i in m.IDXdirectsupply for j in m.IDXproducts)
    return cost - revenue
model3q.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

# Sales of each product cannot exceed their demands
def demand_rule(m, j):
    return m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply) <= m.D[j]
model3q.demand = pe.Constraint(model3q.IDXproducts, rule=demand_rule)

# Maintains the quality of the product within upper bounds
def quality_rule(m, j):
    return m.p * m.y[j] + sum(m.C[i] * m.z[i, j] for i in m.IDXdirectsupply) <= m.PU[j] * (m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply))
model3q.quality = pe.Constraint(model3q.IDXproducts, rule=quality_rule)

# Balances the quality
def balance_quality_rule(m):
    return sum(m.C[i] * m.q[i] * sum(m.y[j] for j in m.IDXproducts) for i in m.IDXpoolsources) == m.p * sum(m.y[j] for j in m.IDXproducts)
model3q.balance_quality = pe.Constraint(rule=balance_quality_rule)

# sum of fraction must be 1
def fraction_rule(m):
    return sum(m.q[i] for i in m.IDXpoolsources) == 1
model3q.fraction = pe.Constraint(rule=fraction_rule)

result_obj = solver.solve(model3q, tee=True)
model3q.pprint()
print(result_obj)

 BARON version 23.6.15. Built: OSX-64 Thu Jun 15 21:44:35 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Can Li at Purdue University, canli@purdue.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSQP
 Doing local search
 Preprocessing found feasible solution with value -100.000
 Solving bounding LP
 Starting multi-start local search
 Preprocessing found feasible solution with value -400.000
 Done with local search
  Iteration    Open nodes         Time (s)    Lower bound      Upper bound
          1             0             0.15     -400.000      

In [27]:
#Q3 PQ formulation
model3pq = pe.ConcreteModel()
model3pq.IDXfeeds = pe.RangeSet(3)
model3pq.IDXpoolsources = pe.RangeSet(2)
model3pq.IDXdirectsupply = {3}
model3pq.IDXproducts = pe.RangeSet(2)
# idx of pools and idx of qualities are just 1
model3pq.c = pe.Param(model3pq.IDXfeeds, initialize=unit_cost)
model3pq.d = pe.Param(model3pq.IDXproducts, initialize=unit_price)
model3pq.C = pe.Param(model3pq.IDXfeeds, initialize=raw_quality)
model3pq.D = pe.Param(model3pq.IDXproducts, initialize=demand)
model3pq.PU = pe.Param(model3pq.IDXproducts, initialize=max_quality)

model3pq.p = pe.Var(within=pe.NonNegativeReals)
# remember no raw mateiral 3 goes to pool
model3pq.q = pe.Var(model3pq.IDXpoolsources, within=pe.NonNegativeReals)
model3pq.y = pe.Var(model3pq.IDXproducts, within=pe.NonNegativeReals)
# remember no raw mateiral 1 and 2 go to direct supply
model3pq.z = pe.Var(model3pq.IDXdirectsupply, model3pq.IDXproducts, within=pe.NonNegativeReals)

def obj_rule(m):
    # term1 = sum((sum(m.q[i] * m.y[j] * m.C[i, j] for i in m.IDXpoolsources)) * y[j] for j in m.IDXproducts)
    # term2 = sum(m.c[i] * m.x[i] for i in m.IDXpoolsources)
    cost =  sum(m.c[i] * (m.q[i] * sum(m.y[j] for j in m.IDXproducts)) for i in m.IDXpoolsources)
    revenue = sum(m.d[j] * m.y[j] for j in m.IDXproducts) + \
                sum((m.d[j] - m.c[i]) * m.z[i,j] for i in m.IDXdirectsupply for j in m.IDXproducts)
    return cost - revenue
model3pq.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

# Sales of each product cannot exceed their demands
def demand_rule(m, j):
    return m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply) <= m.D[j]
model3pq.demand = pe.Constraint(model3pq.IDXproducts, rule=demand_rule)

# Maintains the quality of the product within upper bounds
def quality_rule(m, j):
    return m.p * m.y[j] + sum(m.C[i] * m.z[i, j] for i in m.IDXdirectsupply) <= m.PU[j] * (m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply))
model3pq.quality = pe.Constraint(model3pq.IDXproducts, rule=quality_rule)

# Balances the quality
def balance_quality_rule(m):
    return sum(m.C[i] * m.q[i] * sum(m.y[j] for j in m.IDXproducts) for i in m.IDXpoolsources) == m.p * sum(m.y[j] for j in m.IDXproducts)
model3pq.balance_quality = pe.Constraint(rule=balance_quality_rule)

# sum of fraction must be 1
def fraction_rule(m):
    return sum(m.q[i] for i in m.IDXpoolsources) == 1
model3pq.fraction = pe.Constraint(rule=fraction_rule)

# redundant constraints
def redundant_rule(m, j):
    return sum(m.q[i] * m.y[j] for i in m.IDXpoolsources) <= m.y[j]

result_obj = solver.solve(model3pq, tee=True)
model3pq.pprint()
print(result_obj)

 BARON version 23.6.15. Built: OSX-64 Thu Jun 15 21:44:35 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Can Li at Purdue University, canli@purdue.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSQP
 Doing local search
 Preprocessing found feasible solution with value -100.000
 Solving bounding LP
 Starting multi-start local search
 Preprocessing found feasible solution with value -400.000
 Done with local search
  Iteration    Open nodes         Time (s)    Lower bound      Upper bound
          1             0             0.16     -400.000      